In [ ]:
import requests
import pandas as pd
import typing as T
import os
import json
import time
import numpy as np

from mtgradient.models import DraftTransformer, collate_batch

%load_ext nb_black

In [ ]:
# TODO: read me from config
# set_name = "NEO"
# n_cards = 500

set_name = "HBG"
n_cards = 1000

model_path = os.path.join("artifacts", set_name)
model = DraftTransformer.load_from_checkpoint(
    os.path.join(model_path, "model.ckpt"),
    n_cards=n_cards,
    emb_dim=512,
    n_cards_in_pack=16,
)
model.eval()
with open(os.path.join(model_path, "card_ids.json"), "r") as f:
    card_ids = json.load(f)

In [ ]:
class DraftBot17Lands:
    base_page_url = "https://www.17lands.com/site_draft/{}"
    base_next_url = "https://www.17lands.com/data/site_draft_next/{}"
    base_card_data_url = "https://www.17lands.com/data/site_draft_card_info/{}"
    base_pick_url = "https://www.17lands.com/data/site_draft_make_pick"
    history_url = "https://www.17lands.com/data/site_draft_replay?draft_id={}"
    
    
    def __init__(self, draft_id: str, session_id: str, model: DraftTransformer, card_ids: T.Dict[str, int]):
        self.draft_id = draft_id
        self.session_id = session_id
        self.model = model
        self.card_ids = card_ids
        self.inv_card_ids = {v: k for k, v in card_ids.items()}
        self.finished = False

        r0 = requests.get(self.base_page_url.format(draft_id), cookies={"session": session_id})
        self.cookies = r0.cookies
        self.card_data_raw = requests.get(self.base_card_data_url.format(draft_id), cookies=self.cookies).json()
        
        self.curr_pick = 0
        self.curr_pack = 0
        self.picks = []
        self.next_pack = []
        
        self.history = {}
        self.update_draft()
    
    def update_draft(self):
        """
        Reach out to 17Lands to get the current state of the draft
        """
        self.history = requests.get(self.history_url.format(self.draft_id), cookies=self.cookies).json()
        last = self.history['picks'][-1]
        if last['pick'] is None:
            self.finished = False
        else:
            self.finished = True
    
    
    def make_prediction(self):
        """
        Given the current state of the draft, transform the data
        and run it through the model
        """
        num_hist_rounds = len(self.history["picks"])
        hist = [
            [self.get_card_id(i["name"]) for i in rnd["available"]]
            for rnd in self.history["picks"] if rnd["pick"] is not None
        ]
        
        valid_pick_options = [
            self.get_card_id(i["name"]) for i in self.history["picks"][-1]["available"] 
        ]
        valid_option_names = [
            i["name"] for i in self.history["picks"][-1]["available"] 
        ]
        if len(valid_pick_options) == 0:
            return pd.DataFrame([], columns=["name", "pred"]), [0]
        
        pool = []
        pool_names = []
        for card_set in ("possible_maindeck", "probable_sideboard"):
            cards = self.history["picks"][-1][card_set]
            for color_cards in cards:
                for card in color_cards:
                    pool.append(self.get_card_id(card['name']))

        batch = collate_batch(
            [{"history": hist, "options": valid_pick_options, "pool": pool}],
            device="cpu",
            inference=True,
        )
        card_logits, game_win_pred = model(batch)
        card_probs = np.round(np.exp(card_logits.detach().numpy()[0]), 2)
        pred_df = pd.DataFrame(zip(valid_option_names, card_probs), columns=["name", "pred"])
        return pred_df, game_win_pred.detach().numpy()

    def make_pick(self):
        """
        Sends the selected card id to 17Lands and prints the choice
        """
        pred_df = self.make_prediction()[0]
        pick_name = pred_df.sort_values("pred", ascending=False).iloc[0]["name"]
        pick_id = [k for k, v in self.card_data_raw.items() if v['name'] == pick_name][0]
        r = requests.post(self.base_pick_url, json={"card_id": int(pick_id), "draft_id": self.draft_id}, cookies=self.cookies)
        print(pred_df)
        return r
    
    def get_card_id(self, card_name: str) -> int:
        # adjust for formatting of Alchemy-rebalancing (HBG)
        if card_name[:2] == "A-" and card_name[2:] in self.card_ids:
            return self.card_ids[card_name[2:]]
        return self.card_ids.get(card_name, 0)
    
    def do_draft(self, speed=3):
        n_tries = 0
        while True:
            self.update_draft()
            curr_len = len(self.history["picks"])
            self.make_pick()
            n_tries += 1
            
            if curr_len > 41 or n_tries > 45:
                break
            if len(self.history["picks"][-1]["available"]) == 1 and curr_len > 40:
                break
            time.sleep(speed)


In [ ]:
draft_id = "..."
session = os.environ.get("SESSION_ID", input("session_id:\n"))
draft_bot = DraftBot17Lands(
    draft_id=draft_id, session_id=session, model=model, card_ids=card_ids
)

In [ ]:
draft_bot.do_draft(speed=0.15)

In [ ]:
while True:
    draft_bot.update_draft()
    draft_bot.make_prediction()
    time.sleep(5)